**Sentiment Analysis**

**I. Individual Analysis**

**DVD**

*General Imports*

In [1]:
import nltk
from random import shuffle
from sklearn.linear_model import LogisticRegression
from nltk.metrics.scores import (precision, recall, accuracy)

**Features Construction**

Useful functions to load the different features models

In [2]:
def load_bow(thefeatures,filename):
    '''
    Loads the tags in the filename and appends them to the
    features list.
    '''
    f = open(filename, "r")
    line = f.readline()
    while len(line) >0:
        splited = line.split(' ')
        feats = {}
        for word in range(len(splited)-1):
            theword = splited[word].split(":")
            feats[theword[0]] = int(theword[1])
        tag = splited[-1].split("#:")[1][0:3]
        thefeatures.append((feats,tag))
        line = f.readline()
    f.close()

In [3]:
def load_binary_bow(thefeatures,filename):
    '''
    Loads the tags in the filename and appends them to the
    features list. Filters them to change it to 1 for each 
    existent element in the Bag of Words.
    '''
    f = open(filename, "r")
    line = f.readline()
    while len(line) >0:
        splited = line.split(' ')
        feats = {}
        for word in range(len(splited)-1):
            theword = splited[word].split(":")
            feats[theword[0]] = 1
        tag = splited[-1].split("#:")[1][0:3]
        thefeatures.append((feats,tag))
        line = f.readline()
    f.close()

In [4]:
def load_lexicon(thefeatures,filename):
    '''
    Loads the tags process them to create the Features based on the
    AFINN-111 sentiment lexicon.
    '''
    lex = load_affin()
    f = open(filename, "r")
    line = f.readline()
    while len(line) >0:
        splited = line.split(' ')
        feats1 = {}
        summa = 0
        negga = 0
        pos = 0
        neg = 0
        tot = 0
        
        for word in range(len(splited)-1):
            add = False
            theword = splited[word].split(":")
            lilwords = theword[0].split("_")
            for lw in lilwords:
                if lw in lex:
                    add = True
                    if lex[lw] > 0:
                        summa += lex[lw]*int(theword[1])
                        pos += int(theword[1])
                    else:
                        negga += lex[lw]*int(theword[1])
                        neg += int(theword[1])
            tot += int(theword[1])
        feats1["point_pos"] = summa
        feats1["point_neg"] = -negga
        feats1["count_pos"] = pos
        feats1["count_neg"] = neg
        feats1["size"] = tot
        feats1["porc_pos"] = pos/tot
        feats1["porc_neg"] = neg/tot
        tag = splited[-1].split("#:")[1][0:3]
        thefeatures.append((feats1,tag))
        line = f.readline()
    f.close()

def load_affin():
    '''
    Loads the lexicon of sentiment words included in the AFINN-11 lexicon
    '''
    f = open("AFINN-111.txt", "r")
    line = f.readline()
    affin = {}
    while len(line) >0:
        splited = line.split('\t')
        affin[splited[0]] = int(splited[1])
        line = f.readline()
    return affin

*Additional Function*

Shows the most informative features for the Linear Regression Models.

Taken from the answers to: https://stackoverflow.com/questions/11116697/how-to-get-most-informative-features-for-scikit-learn-classifiers Kudos to him.

In [5]:
def show_most_informative_features(vectorizer, clf, n=20):
    '''
    Gets the most informative features from a Linear Regression Model
    '''
    feature_names = vectorizer.get_feature_names()
    coefs_with_fns = sorted(zip(clf.coef_[0], feature_names))
    top = zip(coefs_with_fns[:n], coefs_with_fns[:-(n + 1):-1])
    for (coef_1, fn_1), (coef_2, fn_2) in top:
        print ("\t%.4f\t%-15s\t\t%.4f\t%-15s" % (coef_1, fn_1, coef_2, fn_2))

**BoW**

Loads the data and creates the feature to train. Loads and uses both review files.

In [6]:
BoWFeatures = []

#Loads the positive reviews
load_bow(BoWFeatures,"reviews/dvd/positive.review")

#Loads the negative reviews 
load_bow(BoWFeatures,"reviews/dvd/negative.review")

print("Loaded",len(BoWFeatures),"reviews")

Loaded 2000 reviews


Loads the data and creates the feature to test the classifier. 

In [7]:
BoWTesting =[]

#Loads the testing reviews 
load_bow(BoWTesting,"reviews/dvd/unlabeled.review")

print("Loaded",len(BoWTesting),"reviews")

Loaded 3586 reviews


**Binary BoW**

Loads the data and creates the feature to train. Loads and uses both review files. Filters them to change it to 1 for each existent element in the Bag of Words.

In [8]:
BinaryBoWFeatures = []

#Loads the positive reviews
load_binary_bow(BinaryBoWFeatures,"reviews/dvd/positive.review")

#Loads the negative reviews 
load_binary_bow(BinaryBoWFeatures,"reviews/dvd/negative.review")

print("Loaded",len(BinaryBoWFeatures),"reviews")

Loaded 2000 reviews


Loads the data and creates the feature to test the classifier. 

In [9]:
BinaryBoWTesting =[]

#Loads the testing reviews 
load_binary_bow(BinaryBoWTesting,"reviews/dvd/unlabeled.review")

print("Loaded",len(BinaryBoWTesting),"reviews")

Loaded 3586 reviews


**Lexicon Features**

Loads the data and creates the feature to train. Loads and uses both review files. Takes into use the following Features:

- Number of positive words
- Number of negative words
- Positive score following the AFINN-111 scores
- Negative score following the AFINN-111 scores
- Size (number of words) of the review
- Percentage of positive words
- Percentage of negative words

In [10]:
LexiconFeatures = []

#Loads the positive reviews
load_lexicon(LexiconFeatures,"reviews/dvd/positive.review")

#Loads the negative reviews 
load_lexicon(LexiconFeatures,"reviews/dvd/negative.review")

print("Loaded",len(LexiconFeatures),"reviews")

Loaded 2000 reviews


Loads the data and creates the feature to test the classifier. 

In [11]:
LexiconTesting =[]

#Loads the testing reviews 
load_lexicon(LexiconTesting,"reviews/dvd/unlabeled.review")

print("Loaded",len(LexiconTesting),"reviews")

Loaded 3586 reviews


**Testing Zone**

Now we are gonna test the six different combinations of Features and classification algorithms

**BoW - Naive Bayes**

Trains the dataset over a Naive Bayes with a Bag of Words Model.

**Training**

In [12]:
shuffle(BoWFeatures)
classifier = nltk.NaiveBayesClassifier.train(BoWFeatures)
classifier.show_most_informative_features(10)

Most Informative Features
                waste_of = 1                 neg : pos    =     33.7 : 1.0
              your_money = 1                 neg : pos    =     23.7 : 1.0
                 a_waste = 1                 neg : pos    =     19.0 : 1.0
                   awful = 1                 neg : pos    =     17.8 : 1.0
             don't_waste = 1                 neg : pos    =     17.7 : 1.0
                  i_felt = 1                 neg : pos    =     17.0 : 1.0
              waste_your = 1                 neg : pos    =     17.0 : 1.0
                   waste = 1                 neg : pos    =     14.5 : 1.0
                   worst = 2                 neg : pos    =     13.7 : 1.0
                    paid = 1                 neg : pos    =     13.0 : 1.0


**Validating**

In [13]:
refsets = nltk.collections.defaultdict(set)
testsets = nltk.collections.defaultdict(set)
real = []
result = []

for i, (feats, label) in enumerate(BoWFeatures):
    refsets[label].add(i)
    observed = classifier.classify(feats)
    testsets[observed].add(i)
    real.append(label)
    result.append(observed)

print( 'For Validating set:')
print( 'Accuracy:', nltk.accuracy(real,result) )
print( 'Precision:', nltk.scores.precision(refsets['pos'], testsets['pos']) )
print( 'Recall:', nltk.recall(refsets['pos'], testsets['pos']) )
print( 'F1 Score:', nltk.f_measure(refsets['pos'], testsets['pos']) )

For Validating set:
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1 Score: 1.0


**Testing**

In [14]:
refsets = nltk.collections.defaultdict(set)
testsets = nltk.collections.defaultdict(set)
real = []
result = []

for i, (feats, label) in enumerate(BoWTesting):
    refsets[label].add(i)
    observed = classifier.classify(feats)
    testsets[observed].add(i)
    real.append(label)
    result.append(observed)

print( 'For testing set:')
print( 'Accuracy:', nltk.accuracy(real,result) )
print( 'Precision:', nltk.scores.precision(refsets['pos'], testsets['pos']) )
print( 'Recall:', nltk.recall(refsets['pos'], testsets['pos']) )
print( 'F1 Score:', nltk.f_measure(refsets['pos'], testsets['pos']) )

For testing set:
Accuracy: 0.8067484662576687
Precision: 0.7585886722376973
Recall: 0.9042612064194798
F1 Score: 0.8250441807624337


**BoW - Linear Regression**

Trains the dataset over a Linear Regression with a Bag of Words Model.

**Training**

In [15]:
shuffle(BoWFeatures)
classifier = nltk.classify.SklearnClassifier(LogisticRegression(max_iter=1000))
classifier.train(BoWFeatures)

<SklearnClassifier(LogisticRegression(max_iter=1000))>

**Validating**

In [16]:
refsets = nltk.collections.defaultdict(set)
testsets = nltk.collections.defaultdict(set)
real = []
result = []

for i, (feats, label) in enumerate(BoWFeatures):
    refsets[label].add(i)
    observed = classifier.classify(feats)
    testsets[observed].add(i)
    real.append(label)
    result.append(observed)

print( 'For Validating set:')
print( 'Accuracy:', nltk.accuracy(real,result) )
print( 'Precision:', nltk.scores.precision(refsets['pos'], testsets['pos']) )
print( 'Recall:', nltk.recall(refsets['pos'], testsets['pos']) )
print( 'F1 Score:', nltk.f_measure(refsets['pos'], testsets['pos']) )

For Validating set:
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1 Score: 1.0


**Testing**

In [17]:
refsets = nltk.collections.defaultdict(set)
testsets = nltk.collections.defaultdict(set)
real = []
result = []

for i, (feats, label) in enumerate(BoWTesting):
    refsets[label].add(i)
    observed = classifier.classify(feats)
    testsets[observed].add(i)
    real.append(label)
    result.append(observed)

print( 'For testing set:')
print( 'Accuracy:', nltk.accuracy(real,result) )
print( 'Precision:', nltk.scores.precision(refsets['pos'], testsets['pos']) )
print( 'Recall:', nltk.recall(refsets['pos'], testsets['pos']) )
print( 'F1 Score:', nltk.f_measure(refsets['pos'], testsets['pos']) )

For testing set:
Accuracy: 0.8259899609592861
Precision: 0.8171581769436997
Recall: 0.8433868289983398
F1 Score: 0.8300653594771241


In [18]:
show_most_informative_features(classifier._vectorizer,classifier._clf,10)  

	-1.0254	bad            		1.0150	great          
	-0.9192	boring         		0.9330	excellent      
	-0.8634	worst          		0.8619	best           
	-0.7481	terrible       		0.6660	love           
	-0.6948	not            		0.6101	loved          
	-0.5971	the_worst      		0.5816	wonderful      
	-0.5938	waste          		0.5744	enjoy          
	-0.5628	no             		0.5281	still          
	-0.5428	stupid         		0.5196	family         
	-0.5425	horrible       		0.5152	must           


**Binary BoW - Naive Bayes**

Trains the dataset over a Naive Bayes with a Binary Bag of Words Model.

**Training**

In [19]:
shuffle(BinaryBoWFeatures)
classifier = nltk.NaiveBayesClassifier.train(BinaryBoWFeatures)
classifier.show_most_informative_features(10)

Most Informative Features
                waste_of = 1                 neg : pos    =     35.0 : 1.0
              your_money = 1                 neg : pos    =     24.3 : 1.0
                   awful = 1                 neg : pos    =     19.8 : 1.0
                 a_waste = 1                 neg : pos    =     19.0 : 1.0
              waste_your = 1                 neg : pos    =     18.3 : 1.0
             don't_waste = 1                 neg : pos    =     17.7 : 1.0
                  i_felt = 1                 neg : pos    =     17.7 : 1.0
                   waste = 1                 neg : pos    =     16.3 : 1.0
                  ruined = 1                 neg : pos    =     15.0 : 1.0
                  a_must = 1                 pos : neg    =     12.4 : 1.0


**Validating**

In [20]:
refsets = nltk.collections.defaultdict(set)
testsets = nltk.collections.defaultdict(set)
real = []
result = []

for i, (feats, label) in enumerate(BinaryBoWFeatures):
    refsets[label].add(i)
    observed = classifier.classify(feats)
    testsets[observed].add(i)
    real.append(label)
    result.append(observed)

print( 'For Validating set:')
print( 'Accuracy:', nltk.accuracy(real,result) )
print( 'Precision:', nltk.scores.precision(refsets['pos'], testsets['pos']) )
print( 'Recall:', nltk.recall(refsets['pos'], testsets['pos']) )
print( 'F1 Score:', nltk.f_measure(refsets['pos'], testsets['pos']) )

For Validating set:
Accuracy: 0.9995
Precision: 0.999000999000999
Recall: 1.0
F1 Score: 0.9995002498750625


**Testing**

In [21]:
refsets = nltk.collections.defaultdict(set)
testsets = nltk.collections.defaultdict(set)
real = []
result = []

for i, (feats, label) in enumerate(BinaryBoWTesting):
    refsets[label].add(i)
    observed = classifier.classify(feats)
    testsets[observed].add(i)
    real.append(label)
    result.append(observed)

print( 'For testing set:')
print( 'Accuracy:', nltk.accuracy(real,result) )
print( 'Precision:', nltk.scores.precision(refsets['pos'], testsets['pos']) )
print( 'Recall:', nltk.recall(refsets['pos'], testsets['pos']) )
print( 'F1 Score:', nltk.f_measure(refsets['pos'], testsets['pos']) )

For testing set:
Accuracy: 0.8134411600669269
Precision: 0.761248852157943
Recall: 0.9175428887659104
F1 Score: 0.832120451693852


**Binary BoW - Linear Regression**

Trains the dataset over a Linear Regression with a Binary Bag of Words Model.

**Training**

In [22]:
shuffle(BinaryBoWFeatures)
classifier = nltk.classify.SklearnClassifier(LogisticRegression(max_iter=1000))
classifier.train(BinaryBoWFeatures)

<SklearnClassifier(LogisticRegression(max_iter=1000))>

**Validating**

In [23]:
refsets = nltk.collections.defaultdict(set)
testsets = nltk.collections.defaultdict(set)
real = []
result = []

for i, (feats, label) in enumerate(BinaryBoWFeatures):
    refsets[label].add(i)
    observed = classifier.classify(feats)
    testsets[observed].add(i)
    real.append(label)
    result.append(observed)

print( 'For Validating set:')
print( 'Accuracy:', nltk.accuracy(real,result) )
print( 'Precision:', nltk.scores.precision(refsets['pos'], testsets['pos']) )
print( 'Recall:', nltk.recall(refsets['pos'], testsets['pos']) )
print( 'F1 Score:', nltk.f_measure(refsets['pos'], testsets['pos']) )

For Validating set:
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1 Score: 1.0


**Testing**

In [24]:
refsets = nltk.collections.defaultdict(set)
testsets = nltk.collections.defaultdict(set)
real = []
result = []

for i, (feats, label) in enumerate(BinaryBoWTesting):
    refsets[label].add(i)
    observed = classifier.classify(feats)
    testsets[observed].add(i)
    real.append(label)
    result.append(observed)

print( 'For testing set:')
print( 'Accuracy:', nltk.accuracy(real,result) )
print( 'Precision:', nltk.scores.precision(refsets['pos'], testsets['pos']) )
print( 'Recall:', nltk.recall(refsets['pos'], testsets['pos']) )
print( 'F1 Score:', nltk.f_measure(refsets['pos'], testsets['pos']) )

For testing set:
Accuracy: 0.8304517568321249
Precision: 0.8180371352785146
Recall: 0.8533480907581626
F1 Score: 0.8353196099674973


In [25]:
show_most_informative_features(classifier._vectorizer,classifier._clf,10)  

	-1.1096	bad            		1.0661	great          
	-0.9211	worst          		0.8730	excellent      
	-0.8783	boring         		0.8320	best           
	-0.8739	not            		0.7464	love           
	-0.7233	terrible       		0.6101	wonderful      
	-0.6712	the_worst      		0.5824	a_great        
	-0.6295	waste          		0.5796	the_best       
	-0.5798	horrible       		0.5687	enjoy          
	-0.5566	no             		0.5515	loved          
	-0.5519	poor           		0.5460	family         


**Lexicon Features - Naive Bayes**

Trains the dataset over a Naive Bayes with a Lexicon Features Model.

**Training**

In [26]:
shuffle(LexiconFeatures)
classifier = nltk.NaiveBayesClassifier.train(LexiconFeatures)
classifier.show_most_informative_features(10)

Most Informative Features
               count_neg = 4                 neg : pos    =      9.0 : 1.0
               count_neg = 26                neg : pos    =      7.0 : 1.0
               point_neg = 14                neg : pos    =      7.0 : 1.0
               count_neg = 13                neg : pos    =      6.3 : 1.0
               count_pos = 16                pos : neg    =      6.3 : 1.0
                porc_pos = 0.07894736842105263    neg : pos    =      5.7 : 1.0
               count_pos = 44                pos : neg    =      5.7 : 1.0
               count_pos = 7                 pos : neg    =      5.7 : 1.0
               point_pos = 117               pos : neg    =      5.7 : 1.0
               point_pos = 120               pos : neg    =      5.7 : 1.0


**Validating**

In [27]:
refsets = nltk.collections.defaultdict(set)
testsets = nltk.collections.defaultdict(set)
real = []
result = []

for i, (feats, label) in enumerate(LexiconFeatures):
    refsets[label].add(i)
    observed = classifier.classify(feats)
    testsets[observed].add(i)
    real.append(label)
    result.append(observed)

print( 'For Validating set:')
print( 'Accuracy:', nltk.accuracy(real,result) )
print( 'Precision:', nltk.scores.precision(refsets['pos'], testsets['pos']) )
print( 'Recall:', nltk.recall(refsets['pos'], testsets['pos']) )
print( 'F1 Score:', nltk.f_measure(refsets['pos'], testsets['pos']) )

For Validating set:
Accuracy: 0.9185
Precision: 0.9131293188548865
Recall: 0.925
F1 Score: 0.9190263288623947


**Testing**

In [28]:
refsets = nltk.collections.defaultdict(set)
testsets = nltk.collections.defaultdict(set)
real = []
result = []

for i, (feats, label) in enumerate(LexiconTesting):
    refsets[label].add(i)
    observed = classifier.classify(feats)
    testsets[observed].add(i)
    real.append(label)
    result.append(observed)

print( 'For testing set:')
print( 'Accuracy:', nltk.accuracy(real,result) )
print( 'Precision:', nltk.scores.precision(refsets['pos'], testsets['pos']) )
print( 'Recall:', nltk.recall(refsets['pos'], testsets['pos']) )
print( 'F1 Score:', nltk.f_measure(refsets['pos'], testsets['pos']) )

For testing set:
Accuracy: 0.6815393195761293
Precision: 0.6920854997111496
Recall: 0.6629773104593248
F1 Score: 0.6772187676653476


**Lexicon Features - Linear Regression**

Trains the dataset over a Linear Regression with a Lexicon Features Model.

**Training**

In [29]:
shuffle(LexiconFeatures)
classifier = nltk.classify.SklearnClassifier(LogisticRegression(max_iter=1000))
classifier.train(LexiconFeatures)

<SklearnClassifier(LogisticRegression(max_iter=1000))>

**Validating**

In [30]:
refsets = nltk.collections.defaultdict(set)
testsets = nltk.collections.defaultdict(set)
real = []
result = []

for i, (feats, label) in enumerate(LexiconFeatures):
    refsets[label].add(i)
    observed = classifier.classify(feats)
    testsets[observed].add(i)
    real.append(label)
    result.append(observed)

print( 'For Validating set:')
print( 'Accuracy:', nltk.accuracy(real,result) )
print( 'Precision:', nltk.scores.precision(refsets['pos'], testsets['pos']) )
print( 'Recall:', nltk.recall(refsets['pos'], testsets['pos']) )
print( 'F1 Score:', nltk.f_measure(refsets['pos'], testsets['pos']) )

For Validating set:
Accuracy: 0.7345
Precision: 0.7390417940876657
Recall: 0.725
F1 Score: 0.7319535588086825


**Testing**

In [31]:
refsets = nltk.collections.defaultdict(set)
testsets = nltk.collections.defaultdict(set)
real = []
result = []

for i, (feats, label) in enumerate(LexiconTesting):
    refsets[label].add(i)
    observed = classifier.classify(feats)
    testsets[observed].add(i)
    real.append(label)
    result.append(observed)

print( 'For testing set:')
print( 'Accuracy:', nltk.accuracy(real,result) )
print( 'Precision:', nltk.scores.precision(refsets['pos'], testsets['pos']) )
print( 'Recall:', nltk.recall(refsets['pos'], testsets['pos']) )
print( 'F1 Score:', nltk.f_measure(refsets['pos'], testsets['pos']) )

For testing set:
Accuracy: 0.7208588957055214
Precision: 0.7233924611973392
Recall: 0.7221914775871611
F1 Score: 0.7227914705067848


In [32]:
show_most_informative_features(classifier._vectorizer,classifier._clf,10)  

	-3.1902	porc_neg       		4.8532	porc_pos       
	-0.0946	count_pos      		0.0531	point_pos      
	-0.0481	point_neg      		0.0336	count_neg      
	0.0027	size           		0.0027	size           
	0.0336	count_neg      		-0.0481	point_neg      
	0.0531	point_pos      		-0.0946	count_pos      
	4.8532	porc_pos       		-3.1902	porc_neg       


**Table of Results - DVD**

| Combination | Accuracy | Precision | Recall | F1 Score|
| --- | ----------- |----------- |----------- |----------- |
| **BOW - NB** | 0.8067484662576687 | 0.7585886722376973 | 0.9042612064194798 | 0.8250441807624337 |
| **BOW - LR** | 0.8259899609592861 | 0.8171581769436997 | 0.8433868289983398 | 0.8300653594771241 |
| **BBOW - NB** | 0.8134411600669269 | 0.761248852157943 | **0.9175428887659104** | 0.832120451693852 |
| **BBOW - LR** | **0.8304517568321249** | **0.8180371352785146** | 0.8533480907581626 | **0.8353196099674973** |
| **Lexicon - NB** | 0.6815393195761293 | 0.6920854997111496 | 0.6629773104593248 | 0.6772187676653476 |
| **Lexicon - LR** | 0.7208588957055214 | 0.7233924611973392 | 0.7221914775871611 | 0.7227914705067848 |
